## Abstracting OSDU environment variables within the ConfigOsduEnvironment class.

At the time of writting this notebook, OSDU has 3 main data partitions which overall work as DEV, TEST, and PROD environmentz.
In addition, depending on the selected OAuth method (Interactive or Non-Interactive) sereral values such as client_id, client_secret, resource_id, scopes, data_partition_id should be used accordingly when instantiating a osdu client to make requests to OSDU.

Let's attempt to define some of the parameters previously mentioned:

    - client_id: The id of the enterprise application registered in the Azure Active Directory, which requires to have access to the service principals of the EQN OSDU instance.
    - client_secret: The generated client secret for the client_id application. If the chosen OAuth method is Interactive, this parameter is not required, while if the method to authenticate is Non-Interactive, the client_secret is required.
    - resource_id: Service pricncipal id. Osdu has 2 main principals: "osdu-main-principle-production" and "osdu-main-principle-non-production".
    - scopes:
    - data-partition-id: Osdu has 3 main data partitions: "npequinor-test", "npequinor-dev", and "equinor-data".


This notebook is intended to abstract this by creating a ConfigOsduEnvironment that provides the correct parameters given the osdu environment.

In [1]:
import os

# Adding .env file variables as environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
class ConfigOsduEnvironment:
    
    """
    Gets the configuration values according to the environment to target
    and the kind of authentication/authorization to be used.
    
    osdu_environment: "npequinor-test", "npequinor-dev", or "equinor-data",
    client_type: "public-client", "private-client", or "token-client".
    The osdu_cli_client (public-client) set as default.
    """
    
    def __init__(self, osdu_environment, client_type = "public-client") -> dict:

        self.osdu_environment = osdu_environment
        self.client_type = client_type
        self.env_variables = self._get_variables
        
    @property
    def _get_variables(self):
        
        self.client_secret = None
        self.refresh_token_key = None
        self.tenant_id = os.environ['tenant_id']
        
        if self.osdu_environment == 'npequinor-dev' or self.osdu_environment == 'npequinor-test':
            
            # osdu
            self.url = os.environ['osdu_np_server']
            self.resource_id = os.environ['np_resource_id']
            if self.osdu_environment == 'npequinor-dev':
                self.data_partition_id = os.environ['osdu_npdev_data_partition_id']
            elif self.osdu_environment == 'npequinor-test':
                self.data_partition_id = os.environ['osdu_nptest_data_partition']
            
            # client data (DEV-TEST)
            if self.client_type=="public-client":
                self.client_id = os.environ['osdu_cli_client_id']
            elif self.client_type=="private-client":
                self.client_id = os.environ['osdu_np_client_id']
                self.client_secret = os.environ['osdu_np_client_secret']
            elif self.client_type=="token-client":
                self.client_id = os.environ['token_client_id']
                self.client_secret = os.environ['token_client_secret']
                self.refresh_token_key = 'token_client_refresh_token_non_prod'

        elif self.osdu_environment == 'equinor-data':
            # osdu
            self.url = os.environ['osdu_prod_server']
            self.resource_id = os.environ['prod_resource_id']
            self.data_partition_id = os.environ['osdu_prod_data_partition_id']
            
            # client data (PROD)
            if self.client_type=="public-client":
                self.client_id = os.environ['osdu_cli_client_id']
            elif self.client_type=="private-client":
                self.client_id = os.environ['osdu_prod_client_id']
                self.client_secret = os.environ['osdu_prod_client_secret']
            elif self.client_type=="token-client":
                self.client_id = os.environ['token_client_id']
                self.client_secret = os.environ['token_client_secret']
                self.refresh_token_key = 'token_client_refresh_token_prod'

        return {
            'osdu_environment': self.osdu_environment,
            'url':self.url,
            'resource_id' : self.resource_id,
            'client_id': self.client_id,
            'client_secret' : self.client_secret,
            'data_partition_id': self.data_partition_id,
            'tenant_id': self.tenant_id,
            'refresh_token_key': self.refresh_token_key            
        }

In [3]:
env = ConfigOsduEnvironment("npequinor-dev", client_type = "public-client")
# env = ConfigOsduEnvironment("npequinor-test", client_type = "public-client")
# env = ConfigOsduEnvironment("equinor-data", client_type = "public-client")

In [4]:
env.env_variables

{'osdu_environment': 'npequinor-dev',
 'url': 'https://npequinor.energy.azure.com/api',
 'resource_id': '7daee810-3f78-40c4-84c2-7a199428de18',
 'client_id': '7a414874-4b27-4378-b34f-bc9e5a5faa4f',
 'client_secret': None,
 'data_partition_id': 'npequinor-dev',
 'tenant_id': '3aa4a235-b6e2-48d5-9195-7fcf05b459b0',
 'refresh_token_key': None}